In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

---
1. Total Scope 1 and 2 Emissions (tons of CO2e): This is the dependent variable that represents the combined direct and indirect greenhouse gas emissions.
2. Charlie (in mil USD): Represents costs incurred on the emission.
3. Romeo (mil USD): Represent production value of emission.
4. Echo (Number): The total number of community members affected.
5. Alpha (mil USD): Total market capital of the company.
---

#### Methodology for Model Development

1. Describe the methodology applied for developing Scope 1 and 2 emission estimation models.
2. Justify the choice of the model/methodology using statistical logic and reasoning.
---
#### Model Results Summary
1. Provide an overview and summary of the results obtained from the developed models.
Model Validation
2. Briefly explain the model validation process.
3. Justify the choice of metrics or validation process used, highlighting their relevance and effectiveness.

---

#### Sector Analysis
1. Identify and analyze which sectors have higher Scope 1 and 2 emissions and which have lower
emissions.
2. Explain the reasons behind the differences in emission levels across sectors.
---
#### Data Requirements for Model Improvement
1. Identify additional data that could improve the accuracy and reliability of the emission
estimation models.
2. Explain how the additional data would enhance the models and lead to better results.o better results.

In [22]:
data_path = r"E:\Workspace\Data Science\Data Science Project\scope_1_scope_2_emmisions\ISSCaseStudy_Sample data for modelling.csv"

df = pd.read_csv(data_path)
df.head(20)

,Company Identifier,Total S1+S2 (tCo2e),Subsector Name,Charlie (milUSD),Alpha (milUSD),Romeo (milUSD),Echo
0,1,6399.0,Software,NaN,NaN,NaN,NaN
1,2,59990.0,Software,1270.020324,1061.630736,8869.070062,21357.0
2,3,296160.0,Specialty Chemicals,6292.864579,2018.705686,10689.439260,34500.0
3,4,960000.0,Specialty Chemicals,2107.656246,2956.230233,3302.499773,5900.0
4,5,253641.0,Health Care Services,8208.619341,1746.387752,11090.105310,61000.0
5,6,5257.0,Diversified Banks,NaN,381.105765,3684.022390,9831.0
6,7,9690000.0,Software,136168.730900,60470.400590,227887.602700,647500.0
7,8,7821702.0,Oil & Gas Exploration & Production,5170.568097,15737.745400,6008.192301,1708.0
8,9,12487.0,Capital Markets,1712.726219,-458.638393,5126.240555,7459.0
9,10,1010030.0,Specialty Chemicals,2724.917214,1874.182466,3694.083713,6000.0


#### Data Preprocessing

- Since there are 872 rows in a dataframe and out of which charlie has the the most number of NaN's followed by 75 in Alpha, 64 in Alpha and 119 in Echo, so dropping the value would significantly result in data loss.
- There are multiple ways to handle NaN's but I am going with filling those NaN's with their respective sectore wise median value

In [23]:
## Nan values per column
df.isna().sum()

Company Identifier       0
Total S1+S2 (tCo2e)      0
Subsector Name           0
Charlie (milUSD)       257
Alpha (milUSD)          75
Romeo (milUSD)          64
Echo                   119
dtype: int64

In [28]:
sector_medians = df.groupby("Subsector Name")[["Charlie (milUSD)",
                             "Alpha (milUSD)","Romeo (milUSD)",
                             "Echo"]].median().T.to_dict()

In [29]:
def fill_missing_values(row):
    sector = row['Subsector Name']
    for col in ["Charlie (milUSD)","Alpha (milUSD)","Romeo (milUSD)","Echo"]:
        if pd.isna(row[col]) and sector in sector_medians:
            row[col] = sector_medians[sector][col]
    return row

df = df.apply(fill_missing_values, axis = 1)

In [30]:
# Create new features
df['Energy_Intensity'] = df['Total S1+S2 (tCo2e)'] / df['Charlie (milUSD)']
df['Emissions_per_Production'] = df['Total S1+S2 (tCo2e)'] / df['Romeo (milUSD)']
df['Raw_Material_Intensity'] = df['Total S1+S2 (tCo2e)'] / df['Alpha (milUSD)']

# Normalize features by sector medians
for sector in sector_medians.keys():
    mask = df['Subsector Name'] == sector
    df.loc[mask, 'Normalized_Charlie'] = df.loc[mask, 'Charlie (milUSD)'] / sector_medians[sector]['Charlie (milUSD)']
    df.loc[mask, 'Normalized_Alpha'] = df.loc[mask, 'Alpha (milUSD)'] / sector_medians[sector]['Alpha (milUSD)']
    df.loc[mask, 'Normalized_Romeo'] = df.loc[mask, 'Romeo (milUSD)'] / sector_medians[sector]['Romeo (milUSD)']
    df.loc[mask, 'Normalized_Echo'] = df.loc[mask, 'Echo'] / sector_medians[sector]['Echo']

# Relative Emission Intensity
sector_emissions_median = df.groupby('Subsector Name')['Total S1+S2 (tCo2e)'].median().to_dict()
df['Relative_Emission_Intensity'] = df.apply(lambda x: x['Total S1+S2 (tCo2e)'] / sector_emissions_median[x['Subsector Name']], axis=1)


In [31]:
df

,Company Identifier,Total S1+S2 (tCo2e),Subsector Name,Charlie (milUSD),Alpha (milUSD),Romeo (milUSD),Echo,Energy_Intensity,Emissions_per_Production,Raw_Material_Intensity,Normalized_Charlie,Normalized_Alpha,Normalized_Romeo,Normalized_Echo,Relative_Emission_Intensity
0,1,6399.00,Software,1095.971939,213.081635,2650.219949,8558.0,5.838653,2.414517,30.030744,1.000000,1.000000,1.000000,1.000000,0.540491
1,2,59990.00,Software,1270.020324,1061.630736,8869.070062,21357.0,47.235465,6.763956,56.507407,1.158807,4.982272,3.346541,2.495560,5.067050
2,3,296160.00,Specialty Chemicals,6292.864579,2018.705686,10689.439260,34500.0,47.062827,27.705850,146.707864,4.746065,2.593629,5.972313,7.436948,1.486859
3,4,960000.00,Specialty Chemicals,2107.656246,2956.230233,3302.499773,5900.0,455.482246,290.688892,324.737901,1.589590,3.798159,1.845145,1.271826,4.819640
4,5,253641.00,Health Care Services,8208.619341,1746.387752,11090.105310,61000.0,30.899350,22.870928,145.237505,2.141880,1.965653,1.591700,1.016667,1.787570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867,868,1423965.78,Oil & Gas Storage & Transportation,3183.102765,2087.455814,3668.409827,3269.5,447.351495,388.169765,682.153735,1.000000,1.000000,1.000000,1.000000,1.324133
868,869,16898.65,Automobiles,993.242228,408.682552,1431.694628,2532.0,17.013624,11.803250,41.349086,0.067831,0.073381,0.090824,0.048321,0.029933
869,870,2983522.00,Coal,1244.507989,1587.863458,1937.988418,0.0,2397.350621,1539.494237,1878.953751,1.000000,0.869234,0.985813,0.000000,1.000000
870,871,2798000.00,Specialty Chemicals,1325.912145,778.332346,5874.240000,4639.0,2110.245396,476.316936,3594.865373,1.000000,1.000000,3.282006,1.000000,14.047243
